<div id="top">包括以下操作：</div>
<li>读取 xml 文件信息, 并返回字典形式。</li>
<li><a href="#1">将 xml 文件 转为 txt 文件形式。</a></li>

### 读取 xml 文件信息, 并返回字典形式

In [ ]:
xmlPath = 'yoloDataTrain/labels/20240102_0.xml'

import xml.etree.ElementTree as ET
tree = ET.parse(xmlPath)
root = tree.getroot() # root <Element 'annotation' at 0x0000027C048D71F0>
print('root', root)

def element_to_dict(element):
    if len(element) == 0: # 遍历到底层，直接返回tag对应的信息
        return {element.tag: element.text}
    
    result = {}
    for child in element:
        if child:
            child_dict = element_to_dict(child)
            if child.tag in result:
                if isinstance(result[child.tag], list):
                    result[child.tag].append(child_dict)
                else:
                    result[child.tag] = [result[child.tag], child_dict]
            else:
                result[child.tag] = child_dict
        else:
            result[child.tag] = child.text
    return result

xml_dict = element_to_dict(root)
print('xml_dict', xml_dict)

### <a id="1">将 xml 文件 转为 txt 文件形式 （2 种方法）</a>

<a href='#top'>▲ Top</a>

#### 法一

在最后由于写入文件的原因是 with open(os.path.join(newdir, filename + '.txt'), 'a+') as f:，a+:追加且可读模式，刚打开时文件指针就在文件末尾，导致文件数量变少，具体原因。。。再试

In [ ]:
import os
import xml.etree.ElementTree as ET
 
dirpath = 'cbw/xmls'  # 存放xml文件的目录
newdir = 'cbw/txtdir' # 形成的txt文件的目录

if not os.path.exists(newdir):
    os.makedirs(newdir)

dict_info = {'H': 0, 'L': 1, 'M': 2}              # 有几个 属性 填写几个label names

i = 0
for fp in os.listdir(dirpath)[:2]:
    if fp.endswith('.xml'):
        i += 1
        root = ET.parse(os.path.join(dirpath, fp)).getroot()
 
        xmin, ymin, xmax, ymax = 0, 0, 0, 0
        sz = root.find('size')
        width = float(sz[0].text)
        height = float(sz[1].text)

        filename = root.find('filename').text
        out_txt_path = os.path.join('cbw/txtdir', filename.split('.')[0] + '.txt').replace('\\','/')
        # print('out_txt_path', out_txt_path)
        out_txt_f = open(out_txt_path, 'w')

        for child in root.findall('object'):  # 找到图片中的所有框
 
            sub = child.find('bndbox')  # 找到框的标注值并进行读取
            label = child.find('name').text
            label_ = dict_info.get(label)
            if label_:
                label_ = label_
            else:
                label_ = 0
            xmin = float(sub[0].text)
            ymin = float(sub[1].text)
            xmax = float(sub[2].text)
            ymax = float(sub[3].text)
            try:  # 转换成yolov3的标签格式，需要归一化到（0-1）的范围内
                x_center = (xmin + xmax) / (2 * width)
                x_center = '%.6f' % x_center
                y_center = (ymin + ymax) / (2 * height)
                y_center = '%.6f' % y_center
                w = (xmax - xmin) / width
                w = '%.6f' % w
                h = (ymax - ymin) / height
                h = '%.6f' % h
            except ZeroDivisionError:
                print(filename, '的 width有问题')
            # print(" fp.split('.xml')[0]",  fp.split('.xml')[0])
            # with open(os.path.join(newdir, fp.split('.xml')[0] + '.txt'), 'a+') as f:  # a+:追加且可读模式，刚打开时文件指针就在文件末尾。
            #     f.write(' '.join([str(label_), str(x_center), str(y_center), str(w), str(h) + '\n']))
            # with open 模式：https://blog.csdn.net/weixin_48681975/article/details/119795252#:~:text=a%2B%3A%E8%BF%BD%E5%8A%A0%E4%B8%94%E5%8F%AF%E8%AF%BB,%E6%8C%87%E9%92%88%E5%B0%B1%E5%9C%A8%E6%96%87%E4%BB%B6%E6%9C%AB%E5%B0%BE%E3%80%82
            
            out_txt_f.write(str(label_) + " " + str(x_center) + str(y_center) + str(w) + str(h) + '\n')

print('ok')
print('i',i)

#### 法二

存在报错原因，是由于将小数点省略了，0.605， 0.726 都变成了 0。

解决方式：.split('.xml') 而不是 .split('.')，<b>这样就可以确保取到完整文件名。 </b>


In [ ]:
import xml.etree.ElementTree as ET
import pickle
import os
from os import listdir, getcwd
from os.path import join

 
def convert(size, box): # 将数值归一化
    # size=(width, height)  b=(xmin, xmax, ymin, ymax)
    # x_center = (xmax+xmin)/2        y_center = (ymax+ymin)/2
    # x = x_center / width            y = y_center / height
    # w = (xmax-xmin) / width         h = (ymax-ymin) / height
 
    x_center = (box[0] + box[1]) / 2.0
    y_center = (box[2] + box[3]) / 2.0
    x = x_center / size[0]
    y = y_center / size[1]
 
    w = (box[1] - box[0]) / size[0]
    h = (box[3] - box[2]) / size[1]
 
    # print(x, y, w, h)
    return (x, y, w, h) # 中心点的坐标 x，y，以及标注框的宽高
 

def convert_annotation(xml_files_path, save_txt_files_path, classes):
    i = 0
    xml_files = os.listdir(xml_files_path)
    # print(xml_files)
    for xml_name in xml_files:
        i += 1
        # print(xml_name)
        xml_file = os.path.join(xml_files_path, xml_name)
        out_txt_path = os.path.join(save_txt_files_path, xml_name.split('.')[0] + '_' + str(i) + '.txt')
        out_txt_f = open(out_txt_path, 'w')
        tree = ET.parse(xml_file)
        root = tree.getroot()
        size = root.find('size')
        w = int(size.find('width').text)
        h = int(size.find('height').text)
 
        for obj in root.iter('object'):
            difficult = obj.find('difficult').text # difficult 是干嘛的
            cls = obj.find('name').text
            if cls not in classes or int(difficult) == 1:
                continue
            cls_id = classes.index(cls)
            xmlbox = obj.find('bndbox')
            b = (float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text),
                 float(xmlbox.find('ymax').text))
            # b=(xmin, xmax, ymin, ymax)
            # print(w, h, b)
            bb = convert((w, h), b)
            out_txt_f.write(str(cls_id) + " " + " ".join([str(a) for a in bb]) + '\n')

    print('i', i)
 
 
if __name__ == "__main__":
    # 把forklift_pallet的voc的xml标签文件转化为yolo的txt标签文件
    # 1、需要转化的类别
    classes = ['H','M','L']
    # 2、voc格式的xml标签文件路径
    xml_files = r'yoloDataTrain/labels'
 
    # 3、转化为yolo格式的txt标签文件存储路径
    save_txt_files = r'yoloDataTrain/txt'
 
    convert_annotation(xml_files, save_txt_files, classes)